In [1]:
import pandas as pd
import os

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import matthews_corrcoef

import altair as alt

In [2]:
### making an assumption that "target actual is the binary value"

eval_results = {}

datasets = ["bace", "clintox", "deepchem_Lipophilicity", "HIV", "sol_del", "tox21"]

data_map={
    'HIV.csv': {'target':'HIV_active','structure':'smiles'},
    'bace.csv':{'target':'active','structure':'mol'},
    'tox21.csv':{'target':'NR-AhR','structure':'smiles'},
    'clintox.csv':{'target':'CT_TOX','structure':'smiles'},
    'sol_del.csv':{'target':'binned_sol','structure':'smiles'},
    'deepchem_Lipophilicity.csv':{'target':'drug_like','structure':'smiles'}   
}

pred_threshold = 0.5

for file in os.scandir(os.path.join("Complex_Models", "Predictions")):
    target = [data_map[dataset]['target'] for dataset in data_map if dataset[:-4] in file.name][0]
    dataset = [dataset for dataset in data_map if dataset[:-4] in file.name][0]
    df = pd.read_csv(file.path)
    y_actual = df["target_actual"].to_list()
    y_pred = df[target].apply(lambda x: 1 if x > pred_threshold else 0).to_list()
    y_pred_prob = df[target].to_list()

    eval_results[file.name] = {
        "dataset" : dataset,
        "accuracy_score": accuracy_score(y_actual, y_pred),
        "balanced_accuracy_score": balanced_accuracy_score(y_actual, y_pred),
        "f1_score": f1_score(y_actual, y_pred),
        "roc_auc_score": roc_auc_score(y_actual, y_pred_prob),
        "log_loss": log_loss(y_actual, y_pred_prob),
        "matthews_corrcoef": matthews_corrcoef(y_actual, y_pred)
    }
    
df = pd.DataFrame(eval_results).T
df = df.reset_index()
df = df.rename(columns = {"index": "result_file"})
df['split_id'] = df['result_file'].apply(lambda x: "train" if "train" in x else "validate")
df['split_method'] = df['result_file'].apply(lambda x: "cluster" if "cluster" in x else "random")
df['dataset'] = df['dataset'].apply(lambda x: "deepchem_lipophilicity.csv" if x == "deepchem_Lipophilicity.csv" else x)
df['matthews_corrcoef']

0     0.970685
1     0.847389
2     0.315091
3     0.176219
4     0.972117
5     0.627845
6      0.87242
7          0.0
8     0.893502
9     0.260579
10    0.943541
11         0.0
12    0.970143
13     0.14481
14    0.707162
15    0.403979
16    0.936691
17    0.508972
18    0.991921
19    0.523379
20    0.984555
21    0.545594
22    0.826429
23    0.334547
24     0.72482
25    0.411727
Name: matthews_corrcoef, dtype: object

In [3]:
# pvt = pd.pivot(df, index = ["dataset", "split_method"], columns = ['split_id'], values = ["accuracy_score", "f1_score", "roc_auc_score", "log_loss"])

# pvt.columns.names = [None, None]

# pvt

In [9]:
# need to come back and optimize the visualizations, but thinking something like multiple facets for each score type
# I also need to rotate text to avoid overlap

vis_df = df.melt(['result_file', 'dataset', 'split_id', 'split_method'], value_name = "score", var_name= "metric")

vis_df['dataset'] = vis_df['dataset'].str[:-4]
vis_df['dataset'] = vis_df['dataset'].str.replace('deepchem_', '')


vis_df.sample(5)



vis_df = vis_df[~ vis_df["result_file"].str.contains("hyperopt")]

### note that I needed to remove hyper-parameter optimizations results
vis_df[(vis_df["dataset"] == 'bace') & (vis_df['split_method'] == "cluster")]




,result_file,dataset,split_id,split_method,metric,score
1,bace-cluster-train-pred.csv,bace,train,cluster,accuracy_score,0.931624
3,bace-cluster-validate-pred.csv,bace,validate,cluster,accuracy_score,0.699115
27,bace-cluster-train-pred.csv,bace,train,cluster,balanced_accuracy_score,0.919875
29,bace-cluster-validate-pred.csv,bace,validate,cluster,balanced_accuracy_score,0.582665
53,bace-cluster-train-pred.csv,bace,train,cluster,f1_score,0.948417
55,bace-cluster-validate-pred.csv,bace,validate,cluster,f1_score,0.802326
79,bace-cluster-train-pred.csv,bace,train,cluster,roc_auc_score,0.981062
81,bace-cluster-validate-pred.csv,bace,validate,cluster,roc_auc_score,0.755303
105,bace-cluster-train-pred.csv,bace,train,cluster,log_loss,0.196036
107,bace-cluster-validate-pred.csv,bace,validate,cluster,log_loss,0.484684


In [10]:
stack = None
chart = None

for i, metric in enumerate(vis_df['metric'].unique()):
    for j, split_type in enumerate(vis_df['split_method'].unique()):
        chart_title = None
        y_title = None
        x_labels = False

        if i == 0: 
            chart_title = "\n" + split_type + " split"

        if i ==0 and j == 0:
            chart_title = ["Summary of All Chemprop Model Results", "", chart_title]
        
        if i == len(vis_df['metric'].unique()) - 1:
            x_labels = True

        if j == 0: y_title = metric

        plot = alt.Chart(vis_df[(vis_df['metric'] == metric) & (vis_df['split_method'] == split_type)]
        ).mark_bar(opacity = 0.7).encode(
                y = alt.Y("score:Q", title = y_title),
                x = alt.X("split_id:N", title= None, axis = alt.Axis(labels = x_labels)),
                color = alt.Color("split_id:N", title = None),
                column =  alt.Row("dataset:N", title = chart_title, header=alt.Header(titleFontSize=14))
                ).properties(width = 30, height = 100)

    
        if not stack:
            stack = plot
        else:
            stack = alt.hconcat(
                stack,
                plot).resolve_scale(
                    y='shared')
            # stack = stack | plot
            # stack.resolve_scale(y = 'shared')

    if not chart:
            chart = stack
    else:
        chart = chart & stack
    stack = None

chart


alt.VConcatChart(...)

In [11]:
### write out dataframe to csv for later comparison with simple model results
vis_df["model"] = 'Chemprop'
vis_df.to_csv(os.path.join("Evaluation", "Chemprop_Results.csv"), index = False)

In [12]:
### Now we just need to pull together the model train time info and look at this

### extract tuples of dataset names and log file paths
log_paths = [( x.name, os.path.join(x.path, "quiet.log")) for x in os.scandir("Complex_Models") if x.name != "Predictions"]
time_data = {
    "model_dataset": {},
    "elapsed_time (hh:mm:ss)": {}
}

i = 0
for dataset, path in log_paths:
    with open(path, 'r') as f:
        elapsed_time = f.readlines()[-1].split('=')[-1].strip()
    
    time_data["model_dataset"][i] = dataset
    time_data["elapsed_time (hh:mm:ss)"][i] = elapsed_time
    
    i += 1

time_df = pd.DataFrame(time_data)

time_df.to_csv(os.path.join("evaluation", "Chemprop_train_time.csv"), index=False)
time_df

### I want to add dataset size in here for some additional context
### Maybe even normalize train time by dataset size?

,model_dataset,elapsed_time (hh:mm:ss)
0,bace,0:03:51
1,bace-cluster,0:04:02
2,bace-cluster-hyperopt,0:05:16
3,bace-random,0:04:04
4,clintox-cluster,0:03:22
5,clintox-random,0:03:21
6,deepchem_Lipophilicity-cluster,0:09:10
7,deepchem_Lipophilicity-random,0:09:33
8,HIV-cluster,4:58:22
9,HIV-random,4:54:50
